In [13]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import random

# 1. Carregar e preparar os dados
file_path = 'data/glass.data'
columns = ['Id', 'RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'Type_of_glass']

df = pd.read_csv(file_path, header=None, names=columns)
df.drop(columns=['Id'], inplace=True)

# Salvar os dados originais para avaliação
df_original = df.copy()

random.seed(100)

# 2. Inserir valores ausentes (MCAR) em Type_of_glass
missing_frac = 0.2
n_missing = int(missing_frac * len(df))
missing_indices = random.sample(list(df.index), n_missing)
df.loc[missing_indices, 'Type_of_glass'] = np.nan


# 1. Separar dados com e sem valores ausentes
df_notnull = df[df['Type_of_glass'].notnull()]
df_null = df[df['Type_of_glass'].isnull()]

# 2. Separar variáveis preditoras (X) e alvo (y)
X_train = df_notnull.drop(columns=['Type_of_glass'])
y_train = df_notnull['Type_of_glass']

X_pred = df_null.drop(columns=['Type_of_glass'])

# 3. Treinar o modelo Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# 4. Imputar os valores ausentes
y_pred = rf.predict(X_pred)

# 5. Comparar com os valores originais
# Índices dos dados com valor ausente
indices_ausentes = df_null.index
valores_reais = df_original.loc[indices_ausentes, 'Type_of_glass']

# 6. Calcular acurácia da imputação
acuracia = accuracy_score(valores_reais, y_pred)
print(f'Taxa de acerto da imputação por Random Forest: {acuracia:.4f}')


Taxa de acerto da imputação por Random Forest: 0.7381


In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import random

# -------------------------------------------------------------
# Função que roda um experimento de imputação usando RF
# para um seed específico
# -------------------------------------------------------------
def run_experiment_rf(seed, missing_frac=0.2):
    
    # 1. Carregar dados
    file_path = 'data/glass.data'
    columns = ['Id', 'RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'Type_of_glass']
    df = pd.read_csv(file_path, header=None, names=columns)
    df.drop(columns=['Id'], inplace=True)

    df_original = df.copy()

    # 2. Inserir valores ausentes MCAR
    random.seed(seed)
    n_missing = int(missing_frac * len(df))
    missing_indices = random.sample(list(df.index), n_missing)
    df.loc[missing_indices, 'Type_of_glass'] = np.nan

    # 3. Separar dados completos e incompletos
    df_notnull = df[df['Type_of_glass'].notnull()]
    df_null = df[df['Type_of_glass'].isnull()]

    X_train = df_notnull.drop(columns=['Type_of_glass'])
    y_train = df_notnull['Type_of_glass']
    X_pred  = df_null.drop(columns=['Type_of_glass'])

    # 4. Treinar o modelo de Random Forest
    rf = RandomForestClassifier(
        n_estimators=200,
        random_state=42
    )
    rf.fit(X_train, y_train)

    # 5. Predição para imputação
    y_pred = rf.predict(X_pred)

    # 6. Comparar com os valores reais removidos
    valores_reais = df_original.loc[df_null.index, 'Type_of_glass']
    acc = accuracy_score(valores_reais, y_pred)

    return acc


# -------------------------------------------------------------
# Executar vários experimentos com seeds variadas
# -------------------------------------------------------------
seeds = list(range(10, 40))  # 30 execuções

results = []

for s in seeds:
    print(f"Rodando seed {s}...")
    acc = run_experiment_rf(s)
    results.append({'seed': s, 'accuracy_rf': acc})

df_results_rf = pd.DataFrame(results)

# Resumo estatístico
summary_rf = df_results_rf.describe().loc[['mean', 'std']]
print("\nResumo dos resultados (Random Forest):")
print(summary_rf)


Rodando seed 10...
Rodando seed 11...
Rodando seed 12...
Rodando seed 13...
Rodando seed 14...
Rodando seed 15...
Rodando seed 16...
Rodando seed 17...
Rodando seed 18...
Rodando seed 19...
Rodando seed 20...
Rodando seed 21...
Rodando seed 22...
Rodando seed 23...
Rodando seed 24...
Rodando seed 25...
Rodando seed 26...
Rodando seed 27...
Rodando seed 28...
Rodando seed 29...
Rodando seed 30...
Rodando seed 31...
Rodando seed 32...
Rodando seed 33...
Rodando seed 34...
Rodando seed 35...
Rodando seed 36...
Rodando seed 37...
Rodando seed 38...
Rodando seed 39...

Resumo dos resultados (Random Forest):
           seed  accuracy_rf
mean  24.500000     0.788095
std    8.803408     0.047351
